In [1]:
# Import necessary libraries
import os
import numpy as np
from sklearn.metrics import f1_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Accuracy, Precision, Recall, AUC
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data Preparation
train_dir = 'Training'
validation_dir = 'Validation'
test_dir = 'Test'

# Data Augmentation and Preprocessing
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

# Specify subdirectories for male and female training data
male_train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(88, 110),
    batch_size=32,
    classes=['male'],  # Specify the class subdirectory
    class_mode='binary'  # Male: 0, Female: 1
)

female_train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(88, 110),
    batch_size=32,
    classes=['female'],  # Specify the class subdirectory
    class_mode='binary'  # Male: 0, Female: 1
)

# Specify subdirectories for male and female test data
male_test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(88, 110),
    batch_size=32,
    classes=['male'],  # Specify the class subdirectory
    class_mode='binary'  # Male: 0, Female: 1
)

female_test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(88, 110),
    batch_size=32,
    classes=['female'],  # Specify the class subdirectory
    class_mode='binary'  # Male: 0, Female: 1
)

# Specify subdirectories for male and female validation data
male_validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(88, 110),
    batch_size=32,
    classes=['male'],  # Specify the class subdirectory
    class_mode='binary'  # Male: 0, Female: 1
)

female_validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(88, 110),
    batch_size=32,
    classes=['female'],  # Specify the class subdirectory
    class_mode='binary'  # Male: 0, Female: 1
)

# Model Architecture (DNN)
model = Sequential([
    Flatten(input_shape=(88, 110, 3)),
    Dense(128, activation='relu'),
    Dropout(0.5),  # Add dropout
    Dense(64, activation='relu'),
    Dropout(0.5),  # Add dropout
    Dense(1, activation='sigmoid')
])

# Model Compilation with F1-Score metric
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=[Accuracy(), Precision(), Recall(), AUC(name='auc')])

# Training
history = model.fit(
    male_train_generator,
    steps_per_epoch=len(male_train_generator),
    epochs=1,  # Adjust the number of epochs as needed
    validation_data=male_validation_generator,
    validation_steps=len(male_validation_generator)
)

# Evaluate on the test data
test_metrics = model.evaluate(male_test_generator)

# Extract test metrics
test_loss = test_metrics[0]
test_accuracy = test_metrics[1]
test_precision = test_metrics[2]
test_recall = test_metrics[3]
test_auc = test_metrics[4]

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")
print(f"Test AUC: {test_auc:.4f}")

# Calculate the F1-Score manually
if test_precision + test_recall == 0:
    test_f1 = 0.0
else:
    test_f1 = 2 * (test_precision * test_recall) / (test_precision + test_recall)

print(f"Test F1-Score: {test_f1:.4f}")


Found 18766 images belonging to 1 classes.
Found 18243 images belonging to 1 classes.
Found 5000 images belonging to 1 classes.
Found 5000 images belonging to 1 classes.
Found 5808 images belonging to 1 classes.
Found 5841 images belonging to 1 classes.
157/157 [==============================] - 12s 79ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.0000e+00
Test Loss: 0.0000
Test Accuracy: 1.0000
Test Precision: 0.0000
Test Recall: 0.0000
Test AUC: 0.0000
Test F1-Score: 0.0000


In [ ]:
model

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# Load the pre-trained model
model = load_model('your_model.keras')  # Replace with the path to your saved Keras model

# Define a function to classify an image
def classify_image(image_path):
    # Load and preprocess the image
    img = load_img(image_path, target_size=(88, 110))
    img_array = img_to_array(img) / 255.0  # Normalize the image
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    # Predict the class (0 for male, 1 for female)
    prediction = model.predict(img_array)

    # Interpret the prediction
    if prediction[0][0] < 0.5:
        return "Male"
    else:
        return "Female"

# Define a function to classify live video from a webcam
def classify_live_video():
    cap = cv2.VideoCapture(0)  # Open the webcam (0 represents the default webcam)

    while True:
        ret, frame = cap.read()  # Read a frame from the webcam

        if not ret:
            break

        # Preprocess the frame (resize and normalize)
        frame = cv2.resize(frame, (88, 110))
        frame = frame / 255.0

        # Make a prediction on the frame
        frame_array = np.expand_dims(frame, axis=0)
        prediction = model.predict(frame_array)

        # Display the result on the frame
        if prediction[0][0] < 0.5:
            cv2.putText(frame, "Male", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        else:
            cv2.putText(frame, "Female", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Display the frame with the prediction
        cv2.imshow('Gender Detection', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the webcam and close all OpenCV windows
    cap.release()
    cv2.destroyAllWindows()

# Example usage for classifying an image
image_path = '131422.jpg'  # Replace with the path to the test image
result = classify_image(image_path)
print(f"Classification Result: {result}")

# Example usage for classifying live video
classify_live_video()
